# Intro

In [202]:
# Goal:
#     - Beating the S&P 500 by Optimally Buying/Selling VOO
#
# Description:
#     - This program aims to outperform the S&P 500 index by strategically buying and selling
#     shares of VOO, an ETF that tracks the S&P 500
#     - The strategy is based on technical indicators to make informed decisions on when to enter and exit positions
#
# Assumptions:
#     - When not invested in VOO, the cash balance does not earn any interest (0% interest rate)
#     - Always buy/sell at close

# Dependencies

In [201]:
!pip install --upgrade ta

In [203]:
import yfinance as yf
import numpy as np
import pandas as pd
import time
import datetime
import plotly.express as px
import ta     # technical analysis library

# Statistical Methods

In [204]:
def percent_change(initial, final):
    return ((final - initial) / initial) * 100

def percent_difference(value1, value2):
    return (value1 - value2) / ((value1 + value2) / 2) * 100

def average(arr):
    return sum(arr) / len(arr) if arr else 0

In [205]:
def BollingerBands(data):
  indicator_bb = ta.volatility.BollingerBands(close=data["Close"], window=21, window_dev=2)

  data['bb_mavg'] = indicator_bb.bollinger_mavg()
  data['bb_Hband'] = indicator_bb.bollinger_hband()
  data['bb_Lband'] = indicator_bb.bollinger_lband()
  data['bb_Hind'] = indicator_bb.bollinger_hband_indicator()    # high indicator
  data['bb_Lind'] = indicator_bb.bollinger_lband_indicator()    # low indicator
  data['bb_w'] = indicator_bb.bollinger_wband()                 # width size bb
  data['bb_p'] = indicator_bb.bollinger_pband()                 # % bb

In [206]:
def MACD(data):
  indicator_mac = ta.trend.MACD(close=data["Close"], window_slow= 26, window_fast= 12, window_sign= 9, fillna = False)

  data['mac'] = indicator_mac.macd()
  data['mac_dif'] = indicator_mac.macd_diff()
  data['mac_ind'] = indicator_mac.macd_signal()

In [207]:
def RSI(data):
  indicator_rsi = ta.momentum.RSIIndicator(close=data["Close"], window = 21, fillna = False)
  indicator_rsi1 = ta.momentum.RSIIndicator(close=data["Close"], window = 100, fillna = False)

  data["rsi"] = indicator_rsi.rsi()
  data['rsi_l'] = indicator_rsi1.rsi()

In [208]:
def SMA(data):
  indicator_sma_l = ta.trend.SMAIndicator(close=data["Close"], window= 200, fillna = False)
  indicator_sma_s = ta.trend.SMAIndicator(close=data["Close"], window= 50, fillna = False)
  indicator_sma_ll = ta.trend.SMAIndicator(close=data["Close"], window= 500, fillna = False)

  data["sma_200"] = indicator_sma_l.sma_indicator()
  data["sma_50"] = indicator_sma_s.sma_indicator()
  data["sma_500"] = indicator_sma_ll.sma_indicator()

# Simulation Helpers

In [209]:
def Performance(results):
  '''Displaying the results'''

  print()
  diffs = list()
  for key, value in results.items():
    key = key.split()

    if "Optimized" in key:
      print(f"Optimized {key[1]} yrs: ${value[0]:.2f}, percentChange: {percent_change(INITIAL_INVESTMENT, value[0]):.2f}%, transactions: {value[1]}")
      dif = percent_difference(value[0], results[f'Baseline {key[1]}'])
      diffs.append(dif)
      print(f"Percent Difference: {dif:.2f}%")
      print()

    else:
      print(f"Baseline {key[1]} yrs: ${value:.2f}, percentChange: {percent_change(INITIAL_INVESTMENT, value):.2f}%")

  return diffs

In [210]:
def Sell(day, data):
  '''Returns True if we should sell VOO on the given day'''

  if (data['rsi'][day] > 70 or data['rsi_l'][day] > 60):  # overbought
    return True

  if (data['sma_50'][day] > data['sma_200'][day]):       # positive deviation from long term
    if (data['Close'][day] < data['bb_Lband'][day]):      # look to position of close
      return True

  elif (data['sma_50'][day] < data['sma_200'][day]):      # negative deviation from long term
    if (data['sma_200'][day] < data['bb_Hband'][day]):      # look to long term avg
      return True

  return False

In [211]:
def Buy(day, data):
  '''Returns True if we should buy VOO on the given day'''

  if (data['rsi'][day] > 70 or data['rsi_l'][day] > 60):    # overbought
    return False

  if (data['sma_50'][day] > data['sma_200'][day]):       # positive deviation from long term
    if (data['Close'][day] > data['bb_Lband'][day]):      # look to position of close
      return True

  elif (data['sma_50'][day] < data['sma_200'][day]):     # negative deviation from long term
    if(data['bb_Hband'][day] < data['sma_50'][day]):      # look to mid term avg
      return True

  return False

# Simulation Setup & Function

In [212]:
TRADING_DAYS_PER_YEAR = 252
DAYS_IN_PAST = 20000
INITIAL_INVESTMENT = 10000.0    # $10,000
TICKER = 'VOO' # s&p 500
YEARS = [.5, 1, 2, 3]
RESULTS = dict()

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=DAYS_IN_PAST)

data = yf.download(TICKER, start=start_date, end=end_date)
data['Date'] = data.index

[*********************100%%**********************]  1 of 1 completed


In [213]:
def MainSimulation(data):
  '''Man sim'''

  # adding columns to data
  BollingerBands(data)
  MACD(data)
  RSI(data)
  SMA(data)

  for years_ago in YEARS:

    start = int (len(data)-years_ago*TRADING_DAYS_PER_YEAR)
    print(years_ago, "years ago")
    print("start:", data.index[start])

    baseline = INITIAL_INVESTMENT
    optimized = INITIAL_INVESTMENT
    isInvested = True                 # start off invested the market
    transactionCount = 0

    for day in range(start, len(data)):
        daily_return = data['Close'][day] / data['Close'][day-1]    # daily return

        if isInvested:                    # is our money in VOO?
          optimized *= daily_return
          if Sell(day, data):             # should we sell at close?
            print(f"Selling at {data['Close'][day]}, {data.index[day]}")
            isInvested = False
            transactionCount+=1

        else:                           # not currently invested
          optimized *= 1              # 0% interest rate
          if Buy(day, data):            # should we buy at close?
            print(f"Buying at {data['Close'][day]}, {data.index[day]}")
            isInvested = True
            transactionCount+=1

        baseline *= daily_return      # update baseline value

    RESULTS[f"Baseline {years_ago}"] = baseline
    RESULTS[f"Optimized {years_ago}"] = optimized, transactionCount
    print()
    print()

# Simulation & Results

In [214]:
MainSimulation(data)

0.5 years ago
start: 2023-11-24 00:00:00
Selling at 432.4800109863281, 2023-12-13 00:00:00
Buying at 430.0899963378906, 2023-12-20 00:00:00
Selling at 451.489990234375, 2024-01-29 00:00:00
Buying at 443.82000732421875, 2024-01-31 00:00:00
Selling at 460.6700134277344, 2024-02-09 00:00:00
Buying at 453.9700012207031, 2024-02-13 00:00:00
Selling at 471.42999267578125, 2024-03-01 00:00:00
Buying at 466.1499938964844, 2024-03-05 00:00:00
Selling at 473.260009765625, 2024-03-07 00:00:00
Buying at 470.3900146484375, 2024-03-08 00:00:00
Selling at 475.0299987792969, 2024-03-12 00:00:00
Buying at 473.2699890136719, 2024-03-14 00:00:00
Selling at 475.6000061035156, 2024-03-19 00:00:00
Buying at 476.6000061035156, 2024-03-26 00:00:00
Selling at 480.760009765625, 2024-03-27 00:00:00
Buying at 476.92999267578125, 2024-04-02 00:00:00
Selling at 463.6099853515625, 2024-04-15 00:00:00
Buying at 459.04998779296875, 2024-04-22 00:00:00


1 years ago
start: 2023-05-25 00:00:00
Selling at 406.75, 2023-06

In [215]:
diffs = Performance(RESULTS)


Baseline 0.5 yrs: $11640.36, percentChange: 16.40%
Optimized 0.5 yrs: $12536.51, percentChange: 25.37%, transactions: 18
Percent Difference: 7.41%

Baseline 1 yrs: $12889.07, percentChange: 28.89%
Optimized 1 yrs: $13934.28, percentChange: 39.34%, transactions: 30
Percent Difference: 7.79%

Baseline 2 yrs: $13342.01, percentChange: 33.42%
Optimized 2 yrs: $16572.33, percentChange: 65.72%, transactions: 36
Percent Difference: 21.60%

Baseline 3 yrs: $12631.84, percentChange: 26.32%
Optimized 3 yrs: $16235.56, percentChange: 62.36%, transactions: 50
Percent Difference: 24.97%



# Data Visualization

In [216]:
fig = px.line(data, x='Date', y=["Close", "sma_200", "sma_50", "bb_Hband", "bb_Lband", "sma_500"])
fig.show()

In [217]:
fig = px.line(data, x='Date', y=["rsi", 'rsi_l'])
fig.show()

In [218]:
fig = px.line(data, x='Date', y=["mac_dif", 'mac', 'mac_ind'])
fig.show()

In [219]:
data

,Open,High,Low,Close,Adj Close,Volume,Date,bb_mavg,bb_Hband,bb_Lband,...,bb_w,bb_p,mac,mac_dif,mac_ind,rsi,rsi_l,sma_200,sma_50,sma_500
Date,,,,,,,,,,,,,,,,,,,,,
2010-09-09,102.500000,102.500000,101.139999,101.320000,78.490959,26500,2010-09-09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-10,101.680000,101.860001,101.300003,101.779999,78.847305,8600,2010-09-10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-13,102.959999,103.139999,102.500000,103.059998,79.838913,33750,2010-09-13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-14,102.839996,103.480003,102.379997,103.040001,79.823402,59400,2010-09-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-15,102.620003,103.379997,102.400002,103.300003,80.024803,9250,2010-09-15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-20,486.779999,488.609985,486.450012,487.170013,487.170013,2599600,2024-05-20,472.930950,491.474383,454.387517,...,7.841920,0.883938,4.864220,1.898761,2.965459,64.294251,58.868078,434.76025,473.079599,396.25708
2024-05-21,486.529999,488.579987,486.350006,488.480011,488.480011,2453600,2024-05-21,474.332380,492.916076,455.748684,...,7.835728,0.880646,5.160963,1.756403,3.404560,65.191690,59.076736,435.15075,473.449199,396.50968
2024-05-22,487.790009,488.429993,484.940002,487.059998,487.059998,3250600,2024-05-22,475.390475,494.220458,456.560493,...,7.921902,0.809865,5.221361,1.453441,3.767920,63.378594,58.750408,435.51565,473.689799,396.75340
